In this chapter we used already fine-tuned BERT models to extract the chunk embeddings of our books. The chunk embeddings correspond to either the meaned embeddings of all the words in the sequence or the embedding of the [CLS] token. We will explore the results of both. We will then run a variety of classifiers over these embeddings directly. 

1.   Meaned pooled output --> single layer NN
2.   RoBERT
3.   ToBERT ?

# Installs, Imports, Configuration

In [1]:
!pip install datasets
!pip install "ray[default]"
!pip install wandb
!pip install tensorboardX

!pip install httplib2==0.15.0
!pip install google-api-python-client==1.6

     |████████████████████████████████| 264 kB 4.2 MB/s 
     |████████████████████████████████| 118 kB 66.3 MB/s 
     |████████████████████████████████| 243 kB 60.9 MB/s 
     |████████████████████████████████| 43 kB 2.9 MB/s 
     |████████████████████████████████| 51.0 MB 63 kB/s 
     |████████████████████████████████| 78 kB 9.5 MB/s 
     |████████████████████████████████| 3.1 MB 62.1 MB/s 
     |████████████████████████████████| 127 kB 68.2 MB/s 
     |████████████████████████████████| 1.3 MB 41.0 MB/s 
     |████████████████████████████████| 72 kB 780 kB/s 
     |████████████████████████████████| 65 kB 5.1 MB/s 
     |████████████████████████████████| 10.1 MB 69.9 MB/s 
     |████████████████████████████████| 201 kB 67.3 MB/s 
     |████████████████████████████████| 85 kB 5.4 MB/s 
     |████████████████████████████████| 142 kB 71.7 MB/s 
     |████████████████████████████████| 294 kB 61.9 MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-py3-none-any.whl size=12617 sha

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from pathlib import Path
import sys
import pickle

In [4]:
import configparser

config = configparser.ConfigParser()
config.read('/content/drive/MyDrive/Thesis/BookSuccessPredictor/config.ini')

drive_base_path = Path(config['Drive']['drive_base_path'])

# sys.path.append(str(drive_base_path / 'BookSuccessPredictor' / '_utils'))
sys.path.append(str(drive_base_path / 'BookSuccessPredictor' / 'datasets' / 'goodreads_maharjan_super' / 'MultiModal' / 'dataset_loader'))

# Get Transformer Model from Stage 1

In [ ]:
import wandb
run = wandb.init()

model_name = 'DistilBERT_multitask_overlap50_dataset_embeddings'

if config['Model']['name'] == 'distilbert-base-uncased':
  if (config['Tokenizer']['overlap']):
    artifact = run.use_artifact('lucaguarro/goodreads_success_predictor/model-nlpbosie:v0', type='model')
  else:
    artifact = run.use_artifact('lucaguarro/goodreads_success_predictor/model-2giwtwvy:v0', type='model')
    
artifact_dir = artifact.download()

transformer_model = DistilBERTForMultipleSequenceClassification.from_pretrained(artifact_dir, num_labels1 = 2, num_labels2 = 8)
model.cuda()

# Getting the Data

## Getting the Pooled Outputs

### Creating the Dataset

#### From script

In [ ]:
import torch as th
import time

def get_book_changes_idx(book_titles):
  book_changes_idx = np.where(np.array(book_titles[:-1]) != np.array(book_titles[1:]))[0]
  book_changes_idx += 1
  book_changes_idx = np.insert(book_changes_idx, 0, 0)
  return book_changes_idx

def getPooledOutputs(model, encoded_dataset, batch_size = 32):
  model.eval()

  # pooled_outputs = []
  pooled_outputs = torch.empty([0,768]).cuda()

  num_iters = (len(encoded_dataset['input_ids']) - 1)//batch_size + 1
  print("total number of iters ", num_iters)
  
  for i in range(num_iters):
    print(i)
    up_to = i*batch_size + batch_size
    if len(encoded_dataset['input_ids']) < up_to:
      up_to = len(encoded_dataset['input_ids'])
    input_ids = th.LongTensor(encoded_dataset['input_ids'][i*batch_size:up_to]).cuda()
    attention_mask = th.LongTensor(encoded_dataset['attention_mask'][i*batch_size:up_to]).cuda()

    with torch.no_grad():
      embeddings = model.forward(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)['hidden_states'][-1][:,0] # Pooled output
      pooled_outputs = th.cat([pooled_outputs, embeddings],0)
      th.cuda.empty_cache()

  return pooled_outputs

In [ ]:
chunked_encoded_dataset

NameError: ignored

In [ ]:
train_set_embeddings = getPooledOutputs(transformer_model, chunked_encoded_dataset['train'])

In [ ]:
val_set_embeddings = getPooledOutputs(transformer_model, chunked_encoded_dataset['validation'])

In [ ]:
test_set_embeddings = getPooledOutputs(transformer_model, chunked_encoded_dataset['test'])

In [ ]:
from datasets import Dataset
train_set_embeddings = Dataset.from_dict({'pooled_outputs': train_set_embeddings})
val_set_embeddings = Dataset.from_dict({'pooled_outputs': val_set_embeddings})
test_set_embeddings = Dataset.from_dict({'pooled_outputs': test_set_embeddings})

In [ ]:
from datasets import concatenate_datasets
dataset_w_embeddings = DatasetDict({
    'train': concatenate_datasets([chunked_encoded_dataset['train'], train_set_embeddings], axis = 1), 
    'validation': concatenate_datasets([chunked_encoded_dataset['validation'], val_set_embeddings], axis = 1), 
    'test': concatenate_datasets([chunked_encoded_dataset['test'], test_set_embeddings], axis = 1)
})
dataset_w_embeddings = dataset_w_embeddings.remove_columns(['attention_mask', 'input_ids', 'token_type_ids'])

In [ ]:
dataset_w_embeddings

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)  

with open('train_ds.pkl', 'wb') as output_file:
  pickle.dump(dataset_w_embeddings['train'], output_file)

with open('val_ds.pkl', 'wb') as output_file:
  pickle.dump(dataset_w_embeddings['validation'], output_file)

with open('test_ds.pkl', 'wb') as output_file:
  pickle.dump(dataset_w_embeddings['test'], output_file)

folder_id = '176pNJFvgTaclx_dKNGgocd-TvwmqxM7Y'
# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': folder_id}]})
file.SetContentFile('train_ds.pkl')
file.Upload() 

# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': folder_id}]})
file.SetContentFile('val_ds.pkl')
file.Upload() 

# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': folder_id}]})
file.SetContentFile('test_ds.pkl')
file.Upload() 

#### Load from Drive

In [5]:
base_path = Path("/content/drive/MyDrive/Thesis/BookSuccessPredictor/datasets/goodreads_maharjan_super/Pooled_Output/60_40/DistilBERT_multitask_sentence_tokenized_dataset_embeddings")

In [6]:
from datasets import DatasetDict
with open(base_path / 'train_ds.pkl', "rb") as input_file:
  train_set_embeddings = pickle.load(input_file)

with open(base_path / 'val_ds.pkl', "rb") as input_file:
  val_set_embeddings = pickle.load(input_file)

with open(base_path / 'test_ds.pkl', "rb") as input_file:
  test_set_embeddings = pickle.load(input_file)

dataset_w_embeddings = DatasetDict({'train': train_set_embeddings, 'validation': val_set_embeddings, 'test': test_set_embeddings})
dataset_w_embeddings

DatasetDict({
    train: Dataset({
        features: ['book_title', 'genre', 'success_label', 'pooled_outputs'],
        num_rows: 14074
    })
    validation: Dataset({
        features: ['book_title', 'genre', 'success_label', 'pooled_outputs'],
        num_rows: 9872
    })
    test: Dataset({
        features: ['book_title', 'genre', 'success_label', 'pooled_outputs'],
        num_rows: 9712
    })
})

## Average Pooled Outputs for Shallow Neural Network and SVM

### Generating the Data from Pooled Outputs

#### From Script

In [ ]:
def getAveragePooledOutputs(model, encoded_dataset):
  book_embeddings_dataset = {'meaned_pooled_output': [], 'book_title': [], 'genre': [], 'labels': []}

  book_changes = get_book_changes_idx(encoded_dataset['book_title'])

  for i in range(len(book_changes)):
    print(i)
    start = book_changes[i]
    end = None
    if i != len(book_changes) - 1:
      end = book_changes[i+1]
    else:
      end = len(encoded_dataset['input_ids'])

    input_ids = th.LongTensor(encoded_dataset['input_ids'][start:end])
    attention_mask = th.BoolTensor(encoded_dataset['attention_mask'][start:end])

    with torch.no_grad():
      embeddings = transformer_model.distilbert(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)[0][:,0] # Pooled output
      book_embeddings = th.mean(embeddings, dim=0) # Takes the mean of the pooled output
    book_embeddings_dataset['meaned_pooled_output'].append(book_embeddings)
    book_embeddings_dataset['book_title'].append(encoded_dataset['book_title'][start])
    book_embeddings_dataset['genre'].append(encoded_dataset['genre'][start])
    book_embeddings_dataset['labels'].append(encoded_dataset['labels'][start])
  
  return book_embeddings_dataset

In [ ]:
avg_pld_outs_ds = getAveragePooledOutputs(dataset_w_embeddings)

In [ ]:
from datasets import DatasetDict, Dataset
avg_pld_outs_hf_ds = DatasetDict({'train': Dataset.from_dict(avg_pld_outs_ds['train']), 'validation': Dataset.from_dict(avg_pld_outs_ds['validation']), 'test': Dataset.from_dict(avg_pld_outs_ds['test'])})

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)  

with open('avg_pld_outs_hf_ds.pkl', 'wb') as output_file:
  pickle.dump(avg_pld_outs_hf_ds, output_file)

folder_id = '176pNJFvgTaclx_dKNGgocd-TvwmqxM7Y'
# get the folder id where you want to save your file
file = drive.CreateFile({'parents':[{u'id': folder_id}]})
file.SetContentFile('avg_pld_outs_hf_ds.pkl')
file.Upload() 

#### Load from Drive

In [ ]:
!pip install datasets

In [ ]:
from datasets import DatasetDict
with open(r"/content/drive/MyDrive/Thesis/BookSuccessPredictor/datasets/goodreads_maharjan_super/Pooled_Output/60_40/DistilBERT_multitask_overlap50_dataset_embeddings/avg_pld_outs_hf_ds.pkl", "rb") as input_file:
  avg_pld_outs_hf_ds = pickle.load(input_file)

# Simple Shallow Neural Network

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers.modeling_outputs import SequenceClassifierOutput

class Net(nn.Module):

    def __init__(self, pre_classifier_init, classifier_init):
        super(Net, self).__init__()

        self.pre_classifier = nn.Linear(768, 768)
        self.classifier = nn.Linear(768, 2)
        self.dropout = nn.Dropout(0.1)

        self.pre_classifier.weight.data.copy_(pre_classifier_init.weight.data)
        self.classifier.weight.data.copy_(classifier_init.weight.data)

        # print(pre_classifier_init.bias.data)
        self.pre_classifier.bias.data.copy_(pre_classifier_init.bias.data)
        self.classifier.bias.data.copy_(classifier_init.bias.data)

        # DOUBLE CHECK IF BIASES ARE BEING SET AS WELL

    def forward(self, x, labels = None):
        # Max pooling over a (2, 2) window
        x = self.pre_classifier(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        logits = self.classifier(x)

        loss = None
        if labels is not None:
          loss_fct = CrossEntropyLoss()
          loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        return SequenceClassifierOutput(
            loss = loss,
            logits = logits
        )

net = Net(model.pre_classifier, model.classifier1)

#### Results with no Training

In [ ]:
net.eval()

In [ ]:
with torch.no_grad():
  logits = net.forward(torch.FloatTensor(avg_pld_outs_hf_ds['validation']['meaned_pooled_output']))
y_score = softmax(logits['logits'], axis = 1)[:, 1].tolist()
y_pred = [math.floor(input) if input < 0.50 else math.ceil(input) for input in y_score]
f1_score(avg_pld_outs_hf_ds['validation']['success_label'], y_pred, average = 'weighted')

In [ ]:
with torch.no_grad():
  logits = net.forward(torch.FloatTensor(avg_pld_outs_hf_ds['test']['meaned_pooled_output']))
y_score = softmax(logits['logits'], axis = 1)[:, 1].tolist()
y_pred = [math.floor(input) if input < 0.50 else math.ceil(input) for input in y_score]
f1_score(avg_pld_outs_hf_ds['test']['success_label'], y_pred, average = 'weighted')

#### Training w Hyperparameter Tuning and Results

In [ ]:
def load_data():
  with open(r"/content/drive/MyDrive/Thesis/Datasets/goodreads_maharjan_super/Pooled_Output/DistilBERT_multitask_overlap50_dataset_embeddings/avg_pld_outs_hf_ds.pkl", "rb") as input_file:
    avg_pld_outs_hf_ds = pickle.load(input_file)
  avg_pld_outs_hf_ds.set_format(type='pt', columns=['meaned_pooled_output', 'success_label'])
  trainset = avg_pld_outs_hf_ds['train']
  valset = avg_pld_outs_hf_ds['validation']
  return trainset, valset

def load_test_data():
  with open(r"/content/drive/MyDrive/Thesis/Datasets/goodreads_maharjan_super/Pooled_Output/DistilBERT_multitask_overlap50_dataset_embeddings/avg_pld_outs_hf_ds.pkl", "rb") as input_file:
    avg_pld_outs_hf_ds = pickle.load(input_file)
  avg_pld_outs_hf_ds.set_format(type='pt', columns=['meaned_pooled_output', 'success_label'])
  testset = avg_pld_outs_hf_ds['test']
  return testset

In [ ]:
from ray import tune
# from ray.tune.integration.wandb import wandb_mixin
# '''@wandb_mixin
# run = wandb.init()

def train_nn(config, checkpoint_dir, data_dir=None):
  net = Net(model.pre_classifier, model.classifier1, config['do_rate'])
  net.train()
  device = "cpu"
  if torch.cuda.is_available():
      device = "cuda:0"
      if torch.cuda.device_count() > 1:
          net = nn.DataParallel(net)
  print(type(net))
  net.to(device)
  # net.cuda()


  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

  trainset, valset = load_data()

  trainloader = torch.utils.data.DataLoader(trainset, batch_size=config["batch_size"], shuffle=True)
  valloader = torch.utils.data.DataLoader(valset, batch_size=config["batch_size"], shuffle=True)

  for epoch in range(config['num_epochs']):
    running_loss = 0.0
    epoch_steps = 0
    for i, data in enumerate(trainloader, 0):

      inputs = data['meaned_pooled_output']
      labels = data['success_label']

      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()

      outputs = net(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item()
      epoch_steps += 1

      if i % 10 == 9:
        print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                        running_loss / epoch_steps))
        running_loss = 0.0

      # Validation loss
      val_loss = 0.0
      val_steps = 0
      total = 0
      correct = 0

      all_predictions = np.array([])
      all_labels = np.array([])

      net.eval()
      with torch.no_grad():
        for i, data in enumerate(valloader, 0):

          inputs_cpu = data['meaned_pooled_output']
          labels_cpu = data['success_label']

          inputs, labels = inputs_cpu.to(device), labels_cpu.to(device)
          # inputs.cuda()
          # labels.cuda()

          outputs = net(inputs)
          _, predicted = torch.max(outputs.data, 1)

          all_predictions = np.append(all_predictions, predicted.to('cpu').numpy())
          all_labels = np.append(all_labels, labels_cpu.numpy())

          total += labels.size(0)
          correct += (predicted == labels).sum().item()

          loss = criterion(outputs, labels)
          val_loss += loss.cpu().numpy()
          val_steps += 1

      with tune.checkpoint_dir(epoch) as checkpoint_dir:
          print("saving in checkpoint dir")
          path = os.path.join(checkpoint_dir, "checkpoint")
          torch.save((net.state_dict(), optimizer.state_dict()), path)

      net.train()

      s_precision, s_recall, s_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')
      # s_acc = accuracy_score(all_labels, all_predictions)
      # wandb.log({"val_loss": val_loss / val_steps, "val_accuracy": correct / total})
      tune.report(loss=(val_loss / val_steps), accuracy=correct / total, f1=s_f1, precision=s_precision, recall=s_recall)
  print("Finished Training")

In [ ]:
def test_results(net, device="cpu"):
    testset = load_test_data()

    testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

    all_predictions = np.array([])
    all_labels = np.array([])

    net.eval()
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            inputs_cpu = data['meaned_pooled_output']
            labels_cpu = data['success_label']

            inputs, labels = inputs_cpu.to(device), labels_cpu.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)

            all_predictions = np.append(all_predictions, predicted.to('cpu').numpy())
            all_labels = np.append(all_labels, labels_cpu.numpy())

    s_precision, s_recall, s_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')
    return {
        'precision': s_precision,
        'recall': s_recall,
        'f1': s_f1
    }

In [ ]:
from ray.tune.logger import DEFAULT_LOGGERS
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.wandb import WandbLoggerCallback
import torch.optim as optim
from functools import partial

def main(num_samples = 15, max_num_epochs = 10):
  config = {
      "lr": tune.loguniform(1e-4, 1e-1),
      "batch_size": tune.choice([16,32,64,128]),
      "num_epochs": tune.choice([1,2]),#,2,3]),#,2,3,5,10,20]),
      "do_rate": tune.uniform(0.1, 0.5),
      "wandb": {
        "project": "AvgPooledOutputClassifier",
        "api_key": config['WandB']['api_key']
      }
    }

  scheduler = ASHAScheduler(
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2)

  result = tune.run(
    partial(train_nn, checkpoint_dir='/tmp/ShallowNNModels'),
    config = config,
    resources_per_trial={'gpu': 1},
    metric = 'loss',
    mode = 'min',
    num_samples = num_samples,
    scheduler = scheduler,
    callbacks=[WandbLoggerCallback(
        project="AvgPooledOutputClassifier",
        group='raytune_hpsearch',
        api_key=config['WandB']['api_key'],
        log_config=True
    )])
  
  best_trial = result.get_best_trial(metric="f1", mode="max", scope="last")
  print("Best trial config: {}".format(best_trial.config))
  print("Best trial final validation loss: {}".format(
      best_trial.last_result["loss"]))
  print("Best trial final validation accuracy: {}".format(
      best_trial.last_result["accuracy"]))
  
  best_trained_model = Net(model.pre_classifier, model.classifier1, best_trial.config['do_rate'])
  device = "cpu"
  if torch.cuda.is_available():
      device = "cuda:0"
      # if gpus_per_trial > 1:
      #     best_trained_model = nn.DataParallel(best_trained_model)
  best_trained_model.to(device)

  best_checkpoint_dir = best_trial.checkpoint.value
  model_state, optimizer_state = torch.load(os.path.join(
      best_checkpoint_dir, "checkpoint"))
  best_trained_model.load_state_dict(model_state)

  # model_save_name = "yungclassifier.pt"
  path = F"/content/drive/MyDrive/Thesis/Models/ShallowNNModels/yungclassifier1.pt"
  torch.save(best_trained_model.state_dict(), path)
  return test_results(best_trained_model, device)

In [ ]:
main(num_samples=15)

# SVM

In [ ]:
from sklearn import svm
import numpy as np

In [ ]:
cs = np.arange(0.9, 1.5, 0.02).tolist()
best_clf = None
best_score = 0
best_c = None
for c in cs:
  clf = svm.SVC(kernel='rbf', gamma='scale', C=c)
  clf.fit(avg_pld_outs_hf_ds['train']['meaned_pooled_output'], avg_pld_outs_hf_ds['train']['success_label'])
  predictions = clf.predict(avg_pld_outs_hf_ds['validation']['meaned_pooled_output'])
  (_, pred_counts) = np.unique(predictions, return_counts=True)
  val_score = f1_score(avg_pld_outs_hf_ds['validation']['success_label'], predictions, average = 'weighted')
  print('Clf with C = {} obtained val-score of {}'.format(c, val_score))
  if (val_score > best_score):
    best_score = val_score
    best_clf = clf
    best_c = c

print('\nBest C: {}; Val-score: {}'.format(best_c, best_score))
test_predictions = best_clf.predict(avg_pld_outs_hf_ds['test']['meaned_pooled_output'])
test_score = f1_score(avg_pld_outs_hf_ds['test']['success_label'], test_predictions, average = 'weighted')
print('Yields score of {} on test set'.format(test_score))

# RoBERT

In [ ]:
dataset_w_embeddings.set_format('pytorch', columns=['pooled_outputs', 'success_label', 'genre'])

In [ ]:
import numpy as np
from datasets import DatasetDict, Dataset

def get_book_changes_idx(book_titles):
  book_changes_idx = np.where(np.array(book_titles[:-1]) != np.array(book_titles[1:]))[0]
  book_changes_idx += 1
  return book_changes_idx

def convert_to_LSTM_dataset_full(dataset):
  full_ds = {}
  full_ds['train'] = convert_to_LSTM_dataset_sub(dataset['train'])
  full_ds['validation'] = convert_to_LSTM_dataset_sub(dataset['validation'])
  full_ds['test'] = convert_to_LSTM_dataset_sub(dataset['test'])

  full_ds = DatasetDict({'train': Dataset.from_dict(full_ds['train']), 'validation': Dataset.from_dict(full_ds['validation']), 'test': Dataset.from_dict(full_ds['test'])})
  return full_ds

def convert_to_LSTM_dataset_sub(dataset):
  ds = {'grouped_pooled_outs': None, 'success_label': None, 'genre': None}

  book_start_idx = get_book_changes_idx(dataset['book_title'])
  book_start_idx_w_end = np.append(book_start_idx, len(dataset['book_title']))
  book_start_idx_w_zero = np.insert(book_start_idx, 0, 0)

  book_lengths = book_start_idx_w_end - np.concatenate((np.array([0]), np.roll(book_start_idx_w_end, 1)[1:]))
  # print(type(dataset['pooled_outputs']))
  book_grouped_embeddings = dataset['pooled_outputs'].split_with_sizes(list(book_lengths))
  # book_grouped_embeddings = torch.stack(dataset['pooled_outputs'].split_with_sizes(list(book_lengths)), dim=0)

  # print(type(book_grouped_embeddings))
  ds['grouped_pooled_outs'] = book_grouped_embeddings
  ds['success_label'] = np.take(dataset['success_label'], book_start_idx_w_zero)
  ds['genre'] = np.take(dataset['genre'], book_start_idx_w_zero)
  return ds

In [ ]:
class RoBERT_Model(nn.Module):

    def __init__(self, layer_size = 100):
        self.layer_size = layer_size
        super(RoBERT_Model, self).__init__()
        self.lstm = nn.LSTM(768, layer_size, num_layers=1, bidirectional=False)
        self.out = nn.Linear(layer_size, 2)

    def forward(self, grouped_pooled_outs):
        """ Define how to performed each call
        Parameters
        __________
        pooled_output: array
            -
        lengt: int
            -
        Returns:
        _______
        -
        """
        # chunks_emb = pooled_out.split_with_sizes(lengt) # splits the input tensor into a list of tensors where the length of each sublist is determined by lengt

        seq_lengths = torch.LongTensor([x for x in map(len, grouped_pooled_outs)]) # gets the length of each sublist in chunks_emb and returns it as an array

        batch_emb_pad = nn.utils.rnn.pad_sequence(grouped_pooled_outs, padding_value=-91, batch_first=True) # pads each sublist in chunks_emb to the largest sublist with value -91
        batch_emb = batch_emb_pad.transpose(0, 1)  # (B,L,D) -> (L,B,D)
        lstm_input = nn.utils.rnn.pack_padded_sequence(batch_emb, seq_lengths, batch_first=False, enforce_sorted=False) # seq_lengths.cpu().numpy()

        packed_output, (h_t, h_c) = self.lstm(lstm_input, )  # (h_t, h_c))
        # output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, padding_value=-91)

        h_t = h_t.view(-1, self.layer_size) # (-1, 100)

        return self.out(h_t) # logits

In [ ]:
def my_collate1(batches):
  # for batch in batches:
  #   print(type(batch['grouped_pooled_outs']), len(batch['grouped_pooled_outs']))
  #   print(type(torch.FloatTensor(batch['grouped_pooled_outs'])))
    return {
        'grouped_pooled_outs': [torch.stack(x['grouped_pooled_outs']) for x in batches],
        'success_label': torch.LongTensor([x['success_label'] for x in batches])
    }

In [ ]:
from transformers import AdamW
import time

def load_test_data():
  full_ds = convert_to_LSTM_dataset_full(dataset_w_embeddings)
  full_ds.set_format(type='torch', columns = ['grouped_pooled_outs', 'success_label', 'genre'])
  testset = full_ds['test']
  return testset

def load_data():
  full_ds = convert_to_LSTM_dataset_full(dataset_w_embeddings)
  full_ds.set_format(type='torch', columns = ['grouped_pooled_outs', 'success_label', 'genre'])
  trainset = full_ds['train']
  valset = full_ds['validation']
  return trainset, valset

# def loss_fun(outputs, targets):
#     loss = nn.CrossEntropyLoss()
#     return loss(outputs, targets)

def rnn_train_fun1(config, checkpoint_dir='/tmp/LSTMModels'):
  model = RoBERT_Model(config["layer_size"])
  model.train()
  device = "cpu"
  # if torch.cuda.is_available():
  #   device = "cuda:0"
  #   if torch.cuda.device_count() > 1:
  #       model = nn.DataParallel(model)
  # # print(type(model))
  # model.to(device)


  criterion = nn.CrossEntropyLoss()
  # optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=0.9)
  optimizer=AdamW(model.parameters(), lr=config["lr"])

  trainset, valset = load_data()

  trainloader = torch.utils.data.DataLoader(trainset, batch_size=config["batch_size"], collate_fn=my_collate1)
  valloader = torch.utils.data.DataLoader(valset, batch_size=config["batch_size"], collate_fn=my_collate1)

  for epoch in range(config['num_epochs']):
    running_loss = 0.0
    epoch_steps = 0

    for batch_idx, batch in enumerate(trainloader):
      grouped_pooled_outs = batch['grouped_pooled_outs'] # .to(device)
      targets = batch['success_label'] #.to(device)

      optimizer.zero_grad()
      outputs = model(grouped_pooled_outs)
      loss = loss_fun(outputs, targets)
      loss.backward()
      model.float()
      optimizer.step()

      running_loss += loss.item()
      epoch_steps += 1

      if batch_idx % 10 == 9:
        print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                        running_loss / epoch_steps))
        running_loss = 0.0

      # Validation loss
      val_loss = 0.0
      val_steps = 0
      total = 0
      correct = 0

      all_predictions = np.array([])
      all_labels = np.array([])

      with torch.no_grad():
          for i, data in enumerate(valloader, 0):

              grouped_pooled_outs = data['grouped_pooled_outs'] # .to(device)
              targets = data['success_label'] # .to(device)

              outputs = model(grouped_pooled_outs)
              _, predicted = torch.max(outputs.data, 1)

              all_predictions = np.append(all_predictions, predicted.numpy())
              all_labels = np.append(all_labels, targets.numpy())

              loss = criterion(outputs, targets)
              val_loss += loss.cpu().numpy()
              val_steps += 1

      with tune.checkpoint_dir(epoch) as checkpoint_dir:
          print("saving in checkpoint dir")
          path = os.path.join(checkpoint_dir, "checkpoint")
          torch.save((model.state_dict(), optimizer.state_dict()), path)

      s_precision, s_recall, s_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')
      tune.report(loss=(val_loss / val_steps), f1=s_f1, precision=s_precision, recall=s_recall)

In [ ]:
def test_results(net, device="cpu"):
  testset = load_test_data()
  testloader = torch.utils.data.DataLoader(testset, batch_size=8, collate_fn=my_collate1)

  all_predictions = np.array([])
  all_labels = np.array([])

  net.eval()
  with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        grouped_pooled_outs = data['grouped_pooled_outs'] # .to(device)
        targets = data['success_label'] # .to(device)

        outputs = net(grouped_pooled_outs)
        _, predicted = torch.max(outputs.data, 1)

        all_predictions = np.append(all_predictions, predicted.numpy())
        all_labels = np.append(all_labels, targets.numpy())

  s_precision, s_recall, s_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')

  return {
      'precision': s_precision,
      'recall': s_recall,
      'f1': s_f1
  }

In [ ]:
from ray.tune.logger import DEFAULT_LOGGERS
from ray.tune.integration.wandb import WandbLogger
from ray.tune.schedulers import ASHAScheduler
from functools import partial

def main(num_samples = 6, max_num_epochs = 15):
  config = {
    "lr": tune.loguniform(5e-4, 5e-2),
    "batch_size": tune.choice([16,32,64]),
    "num_epochs": tune.choice([1,2,3,5]),
    "layer_size": tune.choice([100]),
    "wandb": {
      "project": "LSTMClassifier",
      "api_key": config['WandB']['api_key'],
      "log_config": True
    }
  }

  scheduler = ASHAScheduler(
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2)

  result = tune.run(
    partial(rnn_train_fun1, checkpoint_dir='/tmp/LSTMModels'),
    config = config,
    resources_per_trial={'gpu': 1},
    metric = 'loss',
    mode = 'min',
    num_samples = num_samples,
    scheduler = scheduler,
    callbacks=[WandbLoggerCallback(
        project="LSTMClassifier",
        group='raytune_hpsearch',
        api_key=config['WandB']['api_key'],
        log_config=True
    )])

  
  best_trial = result.get_best_trial(metric="f1", mode="max", scope="last")
  print("Best trial config: {}".format(best_trial.config))
  print("Best trial final validation loss: {}".format(
      best_trial.last_result["loss"]))
  print("Best trial final validation accuracy: {}".format(
      best_trial.last_result["f1"]))
  
  best_trained_model = RoBERT_Model(best_trial.config['layer_size'])
  device = "cpu"
  # if torch.cuda.is_available():
  #     device = "cuda:0"
      # if gpus_per_trial > 1:
      #     best_trained_model = nn.DataParallel(best_trained_model)
  best_trained_model.to(device)
                        
  best_checkpoint_dir = best_trial.checkpoint.value
  model_state, optimizer_state = torch.load(os.path.join(
      best_checkpoint_dir, "checkpoint"))
  best_trained_model.load_state_dict(model_state)

  # model_save_name = "yungclassifier.pt"
  path = F"/content/drive/MyDrive/Thesis/Models/LSTMModels/yungclassifier1.pt"
  torch.save(best_trained_model.state_dict(), path)
  return test_results(best_trained_model, device)

In [ ]:
test_results = main()

# ToBERT

### ToBERT dataset

In [ ]:
dataset_w_embeddings.set_format('pytorch', columns=['pooled_outputs', 'success_label', 'genre'])
book_start_idx, book_lengths = get_book_starts_and_lengths(dataset_w_embeddings['validation']['book_title'])
pld_outs_split = dataset_w_embeddings['validation']['pooled_outputs'].split_with_sizes(list(book_lengths))

In [ ]:
from torch.nn.utils.rnn import pad_sequence
ay = pad_sequence(list(dataset_w_embeddings['validation']['pooled_outputs'].split_with_sizes(list(book_lengths))), batch_first=True)

In [ ]:
book_lengths[1]

43

In [ ]:
torch.set_printoptions(edgeitems=44)

In [ ]:
import numpy as np
from datasets import DatasetDict, Dataset

def get_book_starts_and_lengths(book_titles):
  book_start_idx = get_book_changes_idx(book_titles)
  book_start_idx_w_end = np.append(book_start_idx, len(book_titles))
  book_lengths = book_start_idx_w_end - np.concatenate((np.array([0]), np.roll(book_start_idx_w_end, 1)[1:]))
  return book_start_idx, book_lengths

def get_book_changes_idx(book_titles):
  book_changes_idx = np.where(np.array(book_titles[:-1]) != np.array(book_titles[1:]))[0]
  book_changes_idx += 1
  return book_changes_idx

def convert_to_transformer_dataset_full(dataset):
  full_ds = {}

  # we want to figure out the book_lengths at this point because we want to know how long we should pad the sequences to.
  train_book_start_idx, train_book_lengths = get_book_starts_and_lengths(dataset['train']['book_titles'])
  val_book_start_idx, val_book_lengths = get_book_starts_and_lengths(dataset['validation']['book_titles'])
  test_book_start_idx, test_book_lengths = get_book_starts_and_lengths(dataset['test']['book_titles'])
  max_seq_len = get_max_seq_length(train_book_lengths, val_book_lengths, test_book_lengths)

  full_ds['train'] = convert_to_transformer_dataset_sub(dataset['train'], train_book_start_idx, train_book_lengths, max_seq_len)
  full_ds['validation'] = convert_to_transformer_dataset_sub(dataset['validation'], val_book_start_idx, val_book_lengths, max_seq_len)
  full_ds['test'] = convert_to_transformer_dataset_sub(dataset['test'], test_book_start_idx, test_book_lengths, max_seq_len)

  full_ds = DatasetDict({'train': Dataset.from_dict(full_ds['train']), 'validation': Dataset.from_dict(full_ds['validation']), 'test': Dataset.from_dict(full_ds['test'])})
  return full_ds

def convert_to_transformer_dataset_sub(dataset, book_start_idx, book_lengths, max_seq_length):
  ds = {'grouped_pooled_outs': None, 'success_label': None, 'genre': None}

  # book_lengths = book_start_idx_w_end - np.concatenate((np.array([0]), np.roll(book_start_idx_w_end, 1)[1:]))
  # print(type(dataset['pooled_outputs']))
  book_grouped_embeddings = list(dataset['pooled_outputs'].split_with_sizes(list(book_lengths)))
  book_grouped_embeddigns = pad_sequence(list(book_grouped_embeddings, batch_first=True)
  # book_grouped_embeddings = torch.stack(dataset['pooled_outputs'].split_with_sizes(list(book_lengths)), dim=0)

  book_start_idx_w_zero = np.insert(book_start_idx, 0, 0)
  ds['book_lengths'] = book_lengths
  ds['grouped_pooled_outs'] = book_grouped_embeddings
  ds['success_label'] = torch.take(dataset['success_label'], book_start_idx_w_zero)
  ds['genre'] = torch.take(dataset['genre'], book_start_idx_w_zero)
  return ds

def get_max_seq_length(train_book_lengths, val_book_lengths, test_book_lengths):
  return max(max(train_book_lengths),max(val_book_lengths),max(test_book_lengths))

In [ ]:
dataset_w_embeddings.set_format('pytorch', columns=['pooled_outputs', 'success_label', 'genre'])

In [ ]:
adapted_ds = convert_to_transformer_dataset_full(dataset_w_embeddings)

### Defining Model

The Embedding layer:
nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=0)
uses a vector of size 768. Implies that TransformerEncoderLayer works with embeddings of length 768.

In [ ]:
import torch.nn as nn

# d_model = 768, nhead = 2, d_hid = 200, dropout = 0.1, nlayers = 2
class ToBERT(nn.Module):
    def __init__(
        self,
        d_model=768,
        nhead=2,
        nhid=200,
        num_layers=2,
        dropout=0.1,
        classifier_dropout=0.1,
        # max_len=256,
    ):

        super().__init__()

        # self.d_model = embeddings.size(1)
        assert (
            self.d_model % nhead == 0
        ), "nheads must divide evenly into d_model"

        # self.emb = nn.Embedding.from_pretrained(embeddings, freeze=False)
        # self.pos_encoder = PositionalEncoding(
        #     self.d_model, dropout=dropout, max_len=embeddings.size(0)
        # )

        encoder_layers = nn.TransformerEncoderLayer(
            d_model=self.d_model, nhead=nhead, dim_feedforward=nhid, dropout=dropout, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layers, num_layers=num_layers
        )
        self.classifier = nn.Sequential(
            # Other layers to go here if needed once things seem to be working
            nn.Linear(self.d_model, 2),
        )

    def forward(self, x, src_key_padding_mask):
        # x = self.emb(x) * math.sqrt(self.d_model)
        # x = self.pos_encoder(x)
        x = self.transformer_encoder(x, src_key_padding_mask=src_key_padding_mask)  # self.src_mask)

        # calculates mean taking into account the padding
        x = torch.unsqueeze(1-src_key_padding_mask,2)*ay[0]
        x = x.sum(dim=1)/(1-src_key_padding_mask).sum(dim=1).unsqueeze(1)      
        # x = x.mean(dim=1)
        return self.classifier(x)

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [ ]:
def get_batch_mask(max_seq_len, book_lens):
  mask = torch.zeros(len(book_lens),max_seq_len+1) # batch_size, seq_len
  mask[(torch.arange(len(book_lens)),book_lens)] = 1
  mask = mask.cumsum(dim=1)[:, :-1]
  return mask

In [ ]:
book_lens = torch.LongTensor([2,4,5])
max_seq_len = 6
src_key_padding_mask = get_batch_mask(max_seq_len, book_lens)

In [ ]:
# class ToBERT(nn.Module):

#   def __init__(self, d_model, nhead, dropout, d_hid, nlayers, nclasses):
#       # d_model = 768, nhead = 2, d_hid = 200, dropout = 0.1, nlayers = 2
#       encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
#       self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
#       self.classifier = nn.Linear(d_model, nclasses)

#   def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
#       """
#       Args:
#           src: Tensor, shape [seq_len, embedding_dim, batch_size]
#           src_mask: Tensor, shape [seq_len, seq_len]

#       seq_len should be the max number of segments a book has in our dataset
#       embedding_dim will be 768 (from BERT)

#       src_mask is necessary because we will need to pad shorter books to have as many segments
#       as the longest book. Obviously we do not want our model to attend to the padded tokens in
#       these cases.
#       """
#       output = self.transformer_encoder(src, src_mask) 
#       output = self.classifier(output)
#       return output

### Playground

we can see that the values of the 1st and 2nd tensors didnt change when we applied the masking properly

In [ ]:
def get_batch_mask(max_seq_len, book_lens):
  mask = torch.zeros(len(book_lens),max_seq_len+1) # batch_size, seq_len
  mask[(torch.arange(len(book_lens)),book_lens)] = 1
  mask = mask.cumsum(dim=1)[:, :-1]
  return mask

In [ ]:
book_lens = torch.LongTensor([2,4,5])
max_seq_len = 6
src_key_padding_mask = get_batch_mask(max_seq_len, book_lens)

In [ ]:
src_key_padding_mask.shape

torch.Size([3, 6])

In [ ]:
import torch, torch.nn as nn
q = torch.randn(3, 6, 10) # batch size 3, source sequence length 6, embedding size 10
attn = nn.MultiheadAttention(10, 1, batch_first=True) # embedding size 10, one head

ay = attn(q, q, q, key_padding_mask=src_key_padding_mask) # self attention

In [ ]:
src_key_padding_mask

tensor([[0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.]])

In [ ]:
y = torch.unsqueeze(1-src_key_padding_mask,2)*ay[0]
y.sum(dim=1)/(1-src_key_padding_mask).sum(dim=1).unsqueeze(1)

tensor([[-0.0207,  0.0189, -0.1383,  0.0378, -0.0212, -0.0739, -0.0113,  0.0461,
         -0.0231,  0.1898],
        [-0.0730,  0.2946,  0.1850, -0.4192,  0.3180,  0.5532, -0.2360, -0.2962,
         -0.1943, -0.3110],
        [ 0.2559, -0.1303,  0.3230,  0.0399,  0.0889,  0.0014,  0.0462,  0.0022,
          0.3451, -0.1787]], grad_fn=<DivBackward0>)

# MultiModal

### Defining the Modal

In [ ]:
import torch.nn as nn
import torch

Our model will be composed of three separate modules:

1. (Normalizer) Responsible for taking all the inputs of various dimensions and feeding them each through their own linear layer to project them into a space with all the same dimensions

In essence, it is responsible for eq (1) in the paper $h_i=selu(W_{h_i} x_i + b_h)$


2. (GenreAwareAttention) This is where most of the meat of the model is. It is responsible for performing these 3 equations. 

$score(h_i, g) = v^T selu(W_a h_i + W_g g + b_a)$

$\alpha_i = \frac{exp(score(h_i,g))}{\sum_{i'}exp(score(h_{i'},g)}$

$r=\sum_i \alpha_i h_i$

3. (ClassOutput) The last layer is simply responsible for projecting the book representation to class probabilities.

$\hat{p}=\sigma(W_c r + b_c)$

In [ ]:
class Normalizer(nn.Module):
  def __init__(self, c5g_size, bf_size, std_dims):
    super(Normalizer, self).__init__()

    self.c5g_linear = nn.Linear(c5g_size, std_dims)
    self.bf_linear = nn.Linear(bf_size, std_dims)

  def forward(self, x_c5g, x_bf):
    # x_c5g ~ (BATCH_SIZE, C5G_FEATURE_SIZE)
    # x_bf ~ (BATCH_SIZE, BF_FEATURE_SIZE)
    # # split features into char_5_gram and bert_features
    # char_5_grams = None
    # bert_features = None

    c5g_normed = self.c5g_linear(x_c5g)
    bf_normed = self.bf_linear(x_bf)

    # concatenate c5g_normed and bf_normed
    return torch.stack([c5g_normed, bf_normed], 1) # (BATCH_SIZE, NUM_MODALITIES, EMBED_SIZE)

In [ ]:
class GenreAwareAttention(nn.Module):
  def __init__(self, std_dims, num_units, do_rate):
    super(GenreAwareAttention, self).__init__()
    self.activation = nn.SELU()
    self.nn_softmax = nn.Softmax(dim=1)

    self.v = nn.parameter.Parameter(
        nn.init.xavier_uniform_(torch.empty(num_units,1)),
        requires_grad=True
    )

    self.Wa = nn.parameter.Parameter(
        nn.init.xavier_uniform_(torch.empty(std_dims,num_units)), 
        requires_grad=True
    )

    self.b = nn.parameter.Parameter(
        nn.init.ones_(torch.empty(num_units,)),
        requires_grad=True
    )

    self.Wg = nn.parameter.Parameter(
        nn.init.xavier_uniform_(torch.empty(8, num_units)), 
        requires_grad=True
    )

    self.dropout = nn.Dropout(p=do_rate)

  def forward(self, x, g):
    # x ~ (BATCH_SIZE, NUM_MODALITIES, EMBED_SIZE)
    # g ~ (BATCH_SIZE, 1, GENRE_EMBED_SIZE)
    
    # calculate scores
    atten_g = torch.mm(g, self.Wg).unsqueeze(dim=1)
    et = self.activation(torch.matmul(x, self.Wa) + atten_g + self.b)
    et = self.dropout(et)
    
    et = torch.matmul(et, self.v)

    at = self.nn_softmax(et)

    # at = torch.unsqueeze(at, axis=-1)

    # print('at:', at.size())
    # print('x:', x.size())
    ot = at * x # canot multiply at: torch.Size([4, 2, 1, 1]) x: torch.Size([4, 2, 100])

    return torch.sum(ot, axis=1) # BATCH_SIZE, EMBED_SIZE

In [ ]:
class ClassifierOut(nn.Module):
  def __init__(self, std_dims):
    super(ClassifierOut, self).__init__()
    self.classifier = nn.Linear(std_dims, 2)
  
  def forward(self, r): # r ~ BATCH_SIZE, EMBED_SIZE
    r_out = self.classifier(r) # BATCH_SIZE, 2
    return torch.sigmoid(r_out)

In [ ]:
class FullModel(nn.Module): # may want to consider also adding a dropout layer before classification
  def __init__(self, c5g_size, bf_size, std_dims, num_units, do_rate):
    super(FullModel,self).__init__()
    self.normalizer = Normalizer(c5g_size, bf_size, std_dims)
    self.genre_aware_attention = GenreAwareAttention(std_dims, num_units, do_rate)
    self.classifier_out = ClassifierOut(std_dims)

  def forward(self, x_c5g, x_bf, genre):
    x_normed = self.normalizer(x_c5g, x_bf)
    g_a_a = self.genre_aware_attention(x_normed, genre)
    return self.classifier_out(g_a_a)

### Getting the Data

In [ ]:
from MultimodalGoodreadsDataset import MultimodalGoodreadsDataset

In [ ]:
dataset_base_dir = '/content/drive/MyDrive/Thesis/BookSuccessPredictor/datasets/goodreads_maharjan_super/raw_preprocessed/goodreads_maharjan_trimmed'
cached_features_dir = '/content/drive/MyDrive/Thesis/BookSuccessPredictor/datasets/goodreads_maharjan_super/MultiModal/dataset_loader/cached_features'

ds = MultimodalGoodreadsDataset(dataset_base_dir, cached_features_dir)

def my_collate_fn(batches, f1_len, f2_len):
    return {
        'c5g_f': torch.tensor([x['text_features'].toarray()[0][0:f1_len] for x in batches]), # dtype = Float?
        'bert_f': torch.tensor([x['text_features'].toarray()[0][f1_len:f1_len+f2_len] for x in batches]), 
        'genre': torch.tensor([x['genre'] for x in batches]),
        'label': torch.tensor([x['label'] for x in batches])
    }

# c5g_len = ds.f_lengths[0]
# bf_len = ds.f_lengths[1]

# train_dataloader = DataLoader(ds.train, batch_size=64, shuffle=True, collate_fn=partial(my_collate_fn, f1_len=c5g_len, f2_len=bf_len))
# val_dataloader = DataLoader(ds.val, batch_size=64, shuffle=True, collate_fn=partial(my_collate_fn, f1_len=c5g_len, f2_len=bf_len))
# test_dataloader = DataLoader(ds.test, batch_size=64, shuffle=True, collate_fn=partial(my_collate_fn, f1_len=c5g_len, f2_len=bf_len))

Total test instances: 290, validation instances: 290, and Training instances: 404
Total unique books: 984
Training instances (404,), Val instances (290,), Test instances (290,)
extracting feature: char_5_gram
Using cached features
extracting feature: bert_features
Using cached features


In [ ]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# import pickle

# # 1. Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)  

# with open('train_dataset.pkl', 'wb') as output_file:
#   pickle.dump(ds.train, output_file)

# with open('val_dataset.pkl', 'wb') as output_file:
#   pickle.dump(ds.val, output_file)

# with open('test_dataset.pkl', 'wb') as output_file:
#   pickle.dump(ds.test, output_file)

# folder_id = '1q2IGZrQ9oNwP-CqttWUuiYcenb8vmWUg'
# # get the folder id where you want to save your file
# file = drive.CreateFile({'parents':[{u'id': folder_id}]})
# file.SetContentFile('train_dataset.pkl')
# file.Upload() 

# file = drive.CreateFile({'parents':[{u'id': folder_id}]})
# file.SetContentFile('val_dataset.pkl')
# file.Upload() 

# # get the folder id where you want to save your file
# file = drive.CreateFile({'parents':[{u'id': folder_id}]})
# file.SetContentFile('test_dataset.pkl')
# file.Upload() 

In [ ]:
from datasets import Dataset
class MultimodalGoodreadsDatasetSplit(Dataset):

    def __init__(self, X, genres, Y):
        self.X = X
        self.genres = genres
        self.Y = Y

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        return {'text_features': self.X[idx], 'genre': self.genres[idx], 'label': self.Y[idx]}

mmgrds_train = MultimodalGoodreadsDatasetSplit(ds.train.X, ds.train.genres, ds.train.Y)
mmgrds_val = MultimodalGoodreadsDatasetSplit(ds.val.X, ds.val.genres, ds.val.Y)
mmgrds_test = MultimodalGoodreadsDatasetSplit(ds.test.X, ds.test.genres, ds.test.Y)

In [ ]:
from torch.utils.data import DataLoader
import torch
from functools import partial

def my_collate_fn(batches, f1_len, f2_len):
    return {
        'c5g_f': torch.tensor([x['text_features'].toarray()[0][0:f1_len] for x in batches]), # dtype = Float?
        'bert_f': torch.tensor([x['text_features'].toarray()[0][f1_len:f1_len+f2_len] for x in batches]), 
        'genre': torch.tensor([x['genre'] for x in batches]),
        'label': torch.tensor([x['label'] for x in batches])
    }

def load_data():
  return mmgrds_train, mmgrds_val

def load_test_data():
  return mmgrds_test
# def load_data():
#   dataset_base_dir = '/content/drive/MyDrive/Thesis/BookSuccessPredictor/datasets/goodreads_maharjan_super/raw_preprocessed/goodreads_maharjan_trimmed'
#   cached_features_dir = '/content/drive/MyDrive/Thesis/BookSuccessPredictor/datasets/goodreads_maharjan_super/MultiModal/dataset_loader/cached_features'

#   ds = MultimodalGoodreadsDataset(dataset_base_dir, cached_features_dir)

#   return ds

### Training

In [ ]:
from torch.optim import AdamW
import numpy as np
from sklearn.metrics import precision_recall_fscore_support 

def mm_train_fun1(config, checkpoint_dir='/tmp/MultiModalModels'):

  train_dataset, val_dataset = load_data()
  model = FullModel(311595, 768, config['std_dims'], config['num_units'], config['do_rate']).to('cuda')
  model.train()

  criterion = nn.CrossEntropyLoss()
  optimizer=AdamW(model.parameters(), lr=config["lr"])
  # [311595, 768]
  train_dataloader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, collate_fn=partial(my_collate_fn, f1_len=311595, f2_len=768))
  val_dataloader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=True, collate_fn=partial(my_collate_fn, f1_len=311595, f2_len=768))
  print("len(train_dataloader)",len(train_dataloader))
  for epoch in range(config['num_epochs']):
    running_loss = 0.0
    epoch_steps = 0

    for batch_idx, batch in enumerate(train_dataloader):


      c5g_f = batch['c5g_f'].to('cuda')
      bert_f = batch['bert_f'].to('cuda')
      genre = batch['genre'].to('cuda')
      targets = batch['label'].to('cuda')

      optimizer.zero_grad()
      outputs = model(c5g_f.float(), bert_f.float(), genre.float())
      loss = criterion(outputs, targets)
      loss.backward()
      model.float()
      optimizer.step()

      val_loss = 0.0
      val_steps = 0
      total = 0
      correct = 0

      all_predictions = np.array([])
      all_labels = np.array([])

      model.eval()
      with torch.no_grad():
          for i, batch_v in enumerate(val_dataloader, 0):

              c5g_f = batch_v['c5g_f'].to('cuda')
              bert_f = batch_v['bert_f'].to('cuda')
              genre = batch_v['genre'].to('cuda')
              targets = batch_v['label'].to('cuda')

              outputs = model(c5g_f.float(), bert_f.float(), genre.float())
              _, predicted = torch.max(outputs.data, 1)

              all_predictions = np.append(all_predictions, predicted.cpu().numpy())
              all_labels = np.append(all_labels, targets.cpu().numpy())

              loss = criterion(outputs, targets)
              val_loss += loss.cpu().numpy()
              val_steps += 1

      model.train()
      with tune.checkpoint_dir(epoch) as checkpoint_dir:
          print("saving in checkpoint dir")
          path = os.path.join(checkpoint_dir, "checkpoint")
          torch.save((model.state_dict(), optimizer.state_dict()), path)

      s_precision, s_recall, s_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')
      print('s_precision:', s_precision, 's_recall:', s_recall, 's_f1:', s_f1)
      tune.report(loss = loss.item(), epoch = epoch + batch_idx / len(train_dataloader), eval_loss=(val_loss / val_steps), eval_f1=s_f1, eval_precision=s_precision, eval_recall=s_recall)

In [ ]:
def test_results(net, test_dataloader, device="cpu"):
  all_predictions = np.array([])
  all_labels = np.array([])

  net.to(device)
  net.eval()
  with torch.no_grad():
    for i, batch_test in enumerate(test_dataloader, 0):
        c5g_f = batch_test['c5g_f']
        bert_f = batch_test['bert_f']
        genre = batch_test['genre']
        targets = batch_test['label']

        outputs = net(c5g_f.float(), bert_f.float(), genre.float())
        _, predicted = torch.max(outputs.data, 1)

        all_predictions = np.append(all_predictions, predicted.numpy())
        all_labels = np.append(all_labels, targets.numpy())

  s_precision, s_recall, s_f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')

  return {
      'precision': s_precision,
      'recall': s_recall,
      'f1': s_f1
  }

In [ ]:
from ray import tune
from ray.tune.logger import DEFAULT_LOGGERS
from ray.tune.integration.wandb import WandbLogger, WandbLoggerCallback
from ray.tune.schedulers import ASHAScheduler
from functools import partial
import os

def main(num_samples = 6, max_num_epochs = 15):

  tune_config = {
    "lr": tune.loguniform(5e-4, 5e-2),
    "batch_size": tune.choice([32,64,128]),
    "num_epochs": tune.choice([1]),#,3,5,7,9]),
    "std_dims": tune.sample_from(lambda _: np.random.randint(50,300)),
    "num_units": tune.sample_from(lambda spec: np.random.randint(25,spec.config.std_dims)),
    "do_rate": tune.uniform(0, 0.5),
  }

  scheduler = ASHAScheduler(
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2)

  result = tune.run(
    partial(mm_train_fun1, checkpoint_dir='/tmp/MMModels'),
    config = tune_config,
    resources_per_trial={'gpu': 1},
    metric = 'eval_loss',
    mode = 'min',
    num_samples = num_samples,
    scheduler = scheduler,
    callbacks=[WandbLoggerCallback(
        project="MultiModalClassifier",
        group='raytune_hpsearch',
        api_key=config['WandB']['api_key'],
        log_config=True
    )])

  best_trial = result.get_best_trial(metric="eval_f1", mode="max", scope="last")
  print("Best trial config: {}".format(best_trial.config))
  print("Best trial final validation loss: {}".format(
      best_trial.last_result["eval_loss"]))
  print("Best trial final validation weighted f1: {}".format(
      best_trial.last_result["eval_f1"]))
  
  best_trained_model = FullModel(311595, 768, best_trial.config['std_dims'], best_trial.config['num_units'], best_trial.config['do_rate'])
  device = "cpu"

  best_trained_model.to(device)
                        
  best_checkpoint_dir = best_trial.checkpoint.value
  model_state, optimizer_state = torch.load(os.path.join(
      best_checkpoint_dir, "checkpoint"))
  best_trained_model.load_state_dict(model_state)

  # model_save_name = "yungclassifier.pt"
  path = F"/content/drive/MyDrive/Thesis/BookSuccessPredictor/saved_models/classifier1.pt"
  torch.save(best_trained_model.state_dict(), path)

  test_ds = load_test_data()
  test_dataloader = DataLoader(test_ds, batch_size=best_trial.config["batch_size"], shuffle=True, collate_fn=partial(my_collate_fn, f1_len=311595, f2_len=768))
  return test_results(best_trained_model, test_dataloader, device)

In [ ]:
test_scores = main(num_samples = 1)

2021-08-11 06:44:04,893	WARNING experiment.py:296 -- No name detected on trainable. Using DEFAULT.
2021-08-11 06:44:04,895	INFO registry.py:67 -- Detected unknown callable for trainable. Converting to class.
2021-08-11 06:44:16,757	WARNING worker.py:1189 -- Warning: The actor ImplicitFunc has size 204400654 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2021-08-11 06:44:18,044	WARNING util.py:164 -- The `start_trial` operation took 6.849 s, which may be a performance bottleneck.


Trial name,status,loc,batch_size,do_rate,lr,num_epochs,num_units,std_dims
DEFAULT_874d9_00000,RUNNING,,128,0.260198,0.00192878,1,140,218


(pid=757) len(train_dataloader) 4
(pid=757) saving in checkpoint dir
(pid=757) s_precision: 0.7724137931034483 s_recall: 0.6482758620689655 s_f1: 0.5134804850322092
Result for DEFAULT_874d9_00000:
  date: 2021-08-11_06-45-55
  done: false
  epoch: 0.0
  eval_f1: 0.5134804850322092
  eval_loss: 0.6167731881141663
  eval_precision: 0.7724137931034483
  eval_recall: 0.6482758620689655
  experiment_id: 5548763094fa48b0b0f9cbcb377b9523
  hostname: 3d258191eee7
  iterations_since_restore: 1
  loss: 0.6405212879180908
  node_ip: 172.28.0.2
  pid: 757
  should_checkpoint: true
  time_since_restore: 87.71400427818298
  time_this_iter_s: 87.71400427818298
  time_total_s: 87.71400427818298
  timestamp: 1628664355
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 874d9_00000
  


Trial name,status,loc,batch_size,do_rate,lr,num_epochs,num_units,std_dims,iter,total time (s),loss,epoch,eval_loss
DEFAULT_874d9_00000,RUNNING,172.28.0.2:757,128,0.260198,0.00192878,1,140,218,1,87.714,0.640521,0,0.616773


2021-08-11 06:46:00,112	WARNING util.py:164 -- The `process_trial_save` operation took 4.204 s, which may be a performance bottleneck.


...

2021-08-11 06:49:59,449	WARNING util.py:164 -- The `process_trial_save` operation took 3.918 s, which may be a performance bottleneck.


Trial name,status,loc,batch_size,do_rate,lr,num_epochs,num_units,std_dims,iter,total time (s),loss,epoch,eval_loss
DEFAULT_874d9_00000,TERMINATED,,128,0.260198,0.00192878,1,140,218,4,310.886,0.666941,0.75,0.579312


2021-08-11 06:49:59,673	INFO tune.py:550 -- Total run time: 354.79 seconds (351.12 seconds for the tuning loop).


Best trial config: {'lr': 0.0019287766765341365, 'batch_size': 128, 'num_epochs': 1, 'std_dims': 218, 'num_units': 140, 'do_rate': 0.26019837129864337}
Best trial final validation loss: 0.5793119072914124
Best trial final validation weighted f1: 0.7511382341381775


TypeError: ignored

Learning the genre vectors from Wg, try to understand if some genres are near each other are not using some distance metric (euclidean or manhattan). Can also do PCA.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support 

In [ ]:
s_f1